In [1]:
import duckdb

In [2]:
conn = duckdb.connect()

In [3]:
conn.execute("CREATE TEMP TABLE repo AS SELECT * FROM read_json_auto('./repo_metadata.json');")

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [4]:
query = '''
SELECT nameWithOwner, primaryLanguage, stars, forks, watchers, isFork, isArchived, topics, languages, license
FROM repo
'''

df = conn.execute(query).fetchdf()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [5]:
raw_df = df.copy()

In [6]:
def extract_names(item_ls):
    temp_items = []
    if item_ls is not None and len(item_ls) > 0:
        for item in item_ls:
            temp_items.append(item["name"])
    return temp_items

# Apply the function to the topics column
df['topics'] = df['topics'].apply(extract_names)
df['languages'] = df['languages'].apply(extract_names)

# print(df)

                                             nameWithOwner primaryLanguage  \
0                                freeCodeCamp/freeCodeCamp      TypeScript   
1                   EbookFoundation/free-programming-books            HTML   
2                                     sindresorhus/awesome            None   
3                         codecrafters-io/build-your-own-x        Markdown   
4                                  public-apis/public-apis          Python   
...                                                    ...             ...   
3985963                                   51deep/sucaiwang            Java   
3985964  51deep/SupermarketMembershipPointsManagementSy...            Java   
3985965                          51deep/TeacherClassSystem            Java   
3985966       51deep/TujiaScenicCulturalManagementPlatform            Java   
3985967              51deep/OnlineJapaneseTrainingPlatform            Java   

          stars  forks  watchers  isFork  isArchived  \
0      

In [7]:
# List of topics to filter by
topics_list = [
    "large-language-models",
    "large-language-model",
    "llm",
    "llms",
    "foundation-model",
    "foundation-models",
    "generative-ai",
    "language-model",
    "llm-inference",
    "multimodal-large-language-models",
    'gpt',
    'llama',
    'llama2',
    'openai',
    'chatgpt',
    'generative-ai',
    'multimodal',
    'gpt-4',
    'retrieval-augmented-generation',
    'chain-of-thought',
    'prompt-engineering',
    'langchain',
    'llm-agent'
]

# Query using DuckDB
query = f"""
SELECT * FROM df
WHERE list_has_any(topics, {topics_list})
"""

filtered_df = duckdb.query(query).to_df()

print(filtered_df)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

                          nameWithOwner   primaryLanguage   stars  forks  \
0          Significant-Gravitas/AutoGPT            Python  170526  44831   
1              huggingface/transformers            Python  137641  27570   
2             f/awesome-chatgpt-prompts              HTML  117461  15898   
3                         ollama/ollama                Go  107827   8639   
4               ChatGPTNextWeb/NextChat        TypeScript   78626  60053   
...                                 ...               ...     ...    ...   
17237  sloppylopez/moneypenny-ai-prompt            Kotlin       5      0   
17238                MChatzakis/ChatMGL  Jupyter Notebook       5      0   
17239           ibizabroker/gpt-pdf-bot            Python       5      3   
17240      hivaze/friendly_chatbot_task  Jupyter Notebook       5      1   
17241             Varsha-1605/SocioSell            Python      20     23   

       watchers  isFork  isArchived  \
0          1548   False       False   
1        

In [9]:
filtered_df.to_csv("../temp/llm_kb_github.csv")

In [8]:
filtered_df.head(5)

,nameWithOwner,primaryLanguage,stars,forks,watchers,isFork,isArchived,topics,languages,license
0,Significant-Gravitas/AutoGPT,Python,170526,44831,1548,False,False,"[ai, gpt-4, openai, python, artificial-intelli...","[Python, TypeScript, Dart, C++, CMake, JavaScr...",Other
1,huggingface/transformers,Python,137641,27570,1128,False,False,"[nlp, natural-language-processing, pytorch, la...","[Python, Cuda, Dockerfile, Shell, C++, C, Make...",Apache License 2.0
2,f/awesome-chatgpt-prompts,HTML,117461,15898,1436,False,False,"[chatbot, chatgpt, chatgpt-api, language, bots]","[HTML, Shell, Ruby]",Creative Commons Zero v1.0 Universal
3,ollama/ollama,Go,107827,8639,620,False,False,"[llama, llm, llama2, llms, go, golang, ollama,...","[Go, C, Shell, TypeScript, Makefile, Dockerfil...",MIT License
4,ChatGPTNextWeb/NextChat,TypeScript,78626,60053,420,False,False,"[chatgpt, nextjs, vercel, webui, cross-platfor...","[TypeScript, SCSS, JavaScript, Rust, Shell, Do...",MIT License
